In [3]:
from astropy.table import Table
import pandas as pd
from astropy.cosmology import Planck15 as cosmo
from astropy import units as u
from astropy.io import ascii, fits
from astropy.cosmology import FlatLambdaCDM
import matplotlib.pyplot as plt
import numpy as np

# Loading merged cat:

In [24]:
# # Specify the correct parameters when reading the CSV file
# merged_cat = pd.read_csv('/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/Candelscat/GOODSS/matched_zahra_alex.csv', header=0)
# #

/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_89208/116826839.py:2: DtypeWarning: Columns (679,681,683) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_cat = pd.read_csv('/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/Candelscat/GOODSS/matched_zahra_alex.csv', header=0)


In [43]:
 merged_cat= ascii.read ('/Users/neal/Documents/PhD/RESEARCH/SED-fitting/Clump/matched_zahra_alex.csv')

# Age of univ:

In [11]:
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
def calculate_age_of_universe_at_z(z):
    # Calculate the age of the Universe at redshift z in Gyr
    age = cosmo.age(z).value  # Age in Gyr
    return age

In [12]:
calculate_age_of_universe_at_z(0.5) # testing the age function

8.426346017751783

In [44]:
# converting to pandas
if isinstance(merged_cat, Table):
    merged_cat = merged_cat.to_pandas()

In [45]:
merged_cat.shape

(34930, 757)

In [46]:
print(merged_cat.columns)

Index(['ID', 'IAU_designation', 'RA', 'DEC', 'ID_UV', 'DIST_MATCH',
       'WFC3_F275W_FLUX_OLD', 'WFC3_F275W_FLUXERR_OLD', 'FLAG_COVER',
       'WFC3_F275W_FLUX_IMPROVED',
       ...
       'use_zgrism', 'CLEAR_z', 'Morris_zgrism', 'Morris_quality',
       'Bouwens_zphot', 'McLure_zphot', 'UVUDF_zphot', 'use_z', 'use_z_flag',
       'use_z_err'],
      dtype='object', length=757)


In [47]:
merged_cat = merged_cat[(merged_cat['CLASS_STAR'] <= 0.9)]

In [48]:
merged_cat.shape

(34401, 757)

In [49]:
# Calculate axial ratio q = B_image / A_image and apply filters
merged_cat['A_IMAGE_arcsec'] = merged_cat['A_IMAGE'] * 0.06
merged_cat['B_IMAGE_arcsec'] = merged_cat['B_IMAGE'] * 0.06

merged_cat['axial_ratio'] = merged_cat['B_IMAGE_arcsec'] / merged_cat['A_IMAGE_arcsec']

In [50]:
merged_cat.shape

(34401, 760)

In [51]:
filtered_data = merged_cat[
    (merged_cat['kodra_z_best'] >= 0.5) & (merged_cat['kodra_z_best'] < 3) &
    (merged_cat['CANDELS_F160W'] <= 25) &
    (merged_cat['CANDELS_log_mstar'] >= 9.5) &
    (merged_cat['axial_ratio'] > 0.5) &
    (merged_cat['A_IMAGE_arcsec'] > 0.2)
].copy()  # Added .copy() here

In [52]:
filtered_data.shape

(2818, 760)

In [53]:
#  Calculate SFR and sSFR
filtered_data['SFR'] = 10**filtered_data['CANDELS_log_SFR']  # Assuming candels_log_sfr is log10(SFR)
filtered_data['sSFR'] = (10**filtered_data['SFR']) / (10**filtered_data['CANDELS_log_mstar'])


In [54]:
# Calculate the age of the Universe at each galaxy's redshift and apply the sSFR threshold
filtered_data['tU_z'] = filtered_data['kodra_z_best'].apply(calculate_age_of_universe_at_z)* 1e9 ## Have to convert the units of age from Gyr to yr, bcz the ssfr unit is 1/yr.
filtered_data = filtered_data[filtered_data['sSFR'] > 0.2 / filtered_data['tU_z']]


In [55]:
filtered_data.shape

(2676, 763)

In [56]:

valid_indices= []
for idx, row in filtered_data.iterrows():
    # Check for the first redshift bin and corresponding observation
    if 0.5 <= filtered_data['kodra_z_best'][idx] and filtered_data['kodra_z_best'][idx]< 1 and filtered_data['WFC3_F275W_FLUX_IMPROVED'][idx] != -99:
        valid_indices.append(idx)
    elif 1 <= filtered_data['kodra_z_best'][idx] and filtered_data['kodra_z_best'][idx]< 2 and filtered_data['ACS_F435W_FLUX'][idx] != -99:
        valid_indices.append(idx)
    elif 2 <= filtered_data['kodra_z_best'][idx] and filtered_data['kodra_z_best'][idx]< 3 and filtered_data['ACS_F606W_FLUX'][idx] != -99:
        valid_indices.append(idx)

In [172]:
from astropy.table import Table

# back to an Astropy Table
if isinstance(filtered_data, pd.DataFrame):
    filtered_data_table = Table.from_pandas(filtered_data)


In [178]:
len(filtered_data_table)

2676

In [183]:

valid_indices_table= []
for idx in range(len(filtered_data_table)):
    # Check for the first redshift bin and corresponding observation
    if 0.5 <= filtered_data_table['kodra_z_best'][idx] and filtered_data_table['kodra_z_best'][idx]< 1 and filtered_data_table['WFC3_F275W_FLUX_IMPROVED'][idx] != -99:
        valid_indices_table.append(idx)
    elif 1 <= filtered_data_table['kodra_z_best'][idx] and filtered_data_table['kodra_z_best'][idx]< 2 and filtered_data_table['ACS_F435W_FLUX'][idx] != -99:
        valid_indices_table.append(idx)
    elif 2 <= filtered_data_table['kodra_z_best'][idx] and filtered_data_table['kodra_z_best'][idx]< 3 and filtered_data_table['ACS_F606W_FLUX'][idx] != -99:
        valid_indices_table.append(idx)

In [186]:
print(len(valid_indices_table))
print(valid_indices_table[-5:-1])

2119
[2669, 2670, 2672, 2673]


In [187]:
valid_indices_table =np.asarray(valid_indices_table)
filtered_data_table=filtered_data_table[valid_indices_table]

In [195]:
print(filtered_data_table)

  ID             IAU_designation            ...        tU_z       
----- ------------------------------------- ... ------------------
   18 CANDELS_GDS_F160W_J033224.84-275723.3 ...  2411234228.415048
   31 CANDELS_GDS_F160W_J033224.91-275714.4 ...  2402498254.289286
   40 CANDELS_GDS_F160W_J033221.18-275715.3 ...  5536057423.015836
   49 CANDELS_GDS_F160W_J033221.74-275708.7 ...  3210993224.622638
   57 CANDELS_GDS_F160W_J033225.62-275708.7 ...  7639226316.586187
   73 CANDELS_GDS_F160W_J033223.88-275656.9 ...  4127161320.334082
   79 CANDELS_GDS_F160W_J033221.72-275701.9 ...  5704352551.862238
   97 CANDELS_GDS_F160W_J033220.08-275650.0 ...  2419043535.479037
  107 CANDELS_GDS_F160W_J033220.42-275654.1 ...  7657686708.093629
  135 CANDELS_GDS_F160W_J033234.14-275640.0 ... 2268633729.4320045
  ...                                   ... ...                ...
26728 CANDELS_GDS_F160W_J033234.08-274222.2 ... 4244424049.8812056
26732 CANDELS_GDS_F160W_J033212.47-274221.4 ... 2451701393.593

In [194]:
filtered_data_table.rename_column('ID_1', 'ID')


In [192]:
from astropy.table import Table

# Assuming 'filtered_data_table' is an Astropy Table that has been filtered as per your requirements
# Save the table to a CSV file
filtered_data_table.write('./filtered_data_table_goodss.csv', format='csv', overwrite=True)
